# Use Python function to predict business area and action for car rental company with `ibm-watsonx-ai`

AI function with logic to sequentially call two models: BUSINESS_AREA and ACTION.

You will use **car_rental_training** dataset.


## Learning goals

The learning goals of this notebook are:

-  AI function definition
-  Store AI function
-  Deployment creation


## Contents

This notebook contains the following parts:
1. [Setup](#setup)
2. [Define AI function](#ai_function)
3. [Store the AI function in the repository](#upload)
4. [Deploy AI function](#deploy)
5. [Clean up](#cleanup)
6. [Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).
- Execute notebooks to acquire: <a href="https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20recommend%20mitigation%20for%20car%20rental%20company.ipynb" target="_blank" rel="noopener no referrer">CARS4U - Action Recommendation model deployment</a> &  <a href="https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20predict%20business%20area%20for%20car%20rental%20company.ipynb" target="_blank" rel="noopener no referrer">CARS4U - Business Area Prediction Model deployment</a>

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to watsonx.ai Runtime

Authenticate the watsonx.ai Runtime service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the watsonx.ai Runtime docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [29]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [1]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url='https://' + location + '.ml.cloud.ibm.com'
)

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="ai_function"></a>
## 2. Define AI function

In this section we define AI function and test it locally.

### Template

We start with template version of AI function to present its structure in simplest case.

    # Template version

    def score_template(payload):
        """AI function example.

        # Here we present data schema to be used by function:
        Example:
        {
          "input_data":
              [{
                "fields": ["Gender", "Status", "Children", "Age", "Customer_Status"],
                "values": [
                  ["Male", "M", 2, 48, "Inactive"],
                  ["Female", "S", 0, 23, "Inactive"]
                ]
              }]
        }
        """
        fields = payload['input_data'][0]['fields'] + ['Prediction', 'Probability']
        values = [record + [int(record[0] == 'Male'), 0.9] for record in payload['input_data'][0]['values']]
        return { 'fields': fields, 'values': values }

    template_payload = {'input_data': [{"fields": [],
                        "values": [["Male", "M", 2, 48, "Inactive"], ["Female", "S", 0, 23, "Inactive"]]}]}

    print(score_template(template_payload))

### Deployed models metadata extraction

We have to identify model deployments to be used before we define AI function. We extract scoring url and information about model version.

We list all deployed models predicting `Action` and `Business Area` to select one to be used in the AI function.

In [ ]:
client.deployments.list()

We select deployed model to predict `Business Area` and model to recommend `Action`. We have to extract deployments IDs.

**Tip**: If you don't have required deployments ready yet, please refer to the following notebooks to prepare them:
- [Bisnes area](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20predict%20business%20area%20for%20car%20rental%20company.ipynb)
- [Action Recommendation](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20recommend%20mitigation%20for%20car%20rental%20company.ipynb)

In [12]:
deployments = client.deployments.get_details()['resources']

for x in deployments:
    if (x['entity']['name'] == 'CARS4U - Action Recommendation model deployment'):
        action_deployment_id = x['metadata']['id']
    if (x['entity']['name'] == 'CARS4U - Business Area Prediction Model deployment'):
        business_area_deployment_id = x['metadata']['id']
        
print(f"Action deployment id: {action_deployment_id}")
print(f"Business area deployment id: {business_area_deployment_id}")

Action deployment id: 23d96ff2-2e23-4e58-b944-c6c9400b8227
Business area deployment id: 23c80ac9-c244-4e67-8c3f-1743e4e958e7


### AI function definition

We define AI function to be deployed in the IBM Cloud.

Put all parameters required to define AI function in dictionary.

In [13]:
ai_params = {"credentials": credentials, 
             "space_id": space_id,
             "action_deployment_id": action_deployment_id, 
             "business_area_deployment_id": business_area_deployment_id}

In [14]:
def score_generator(params=ai_params):
    
    from ibm_watsonx_ai import APIClient

    credentials = params["credentials"]
    space_id = params['space_id']
    action_deployment_id = params["action_deployment_id"]
    business_area_deployment_id = params["business_area_deployment_id"]

    client = APIClient(credentials)
    client.set.default_space(space_id)

    def score(payload):
        """AI function with model version.

        Example:
          {"fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
           "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]}
        """
        
        fields = payload["input_data"][0]['fields']
        
        scores_area = client.deployments.score(business_area_deployment_id, payload).get('predictions', [])
        idx_area = [i for i, x in enumerate(scores_area[0]["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        fields_from_action = fields + ["predictedLabel"]
        
        idx_from_action = set([i for i, x in enumerate(scores_area[0]["fields"]) if x in fields_from_action])
        values_action = [[v for i, v in enumerate(rec) if i in idx_from_action] for rec in scores_area[0]["values"]]
        
        payload_action = {
            "input_data": [
                {
                    "fields": fields + ["Business_Area"], 
                    "values": values_action
                }
            ]
        }
        
        scores_action = client.deployments.score(action_deployment_id, payload_action).get('predictions', [])
        idx_action = [i for i, x in enumerate(scores_action[0]["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        values = [rec + [scores_area[0]['values'][i][idx_area[0]], scores_area[0]['values'][i][idx_area[1]], scores_action[0]['values'][i][idx_action[0]], scores_action[0]['values'][i][idx_action[1]]] for i, rec in enumerate(payload["input_data"][0]['values'])]

        fields = fields + ['Probability_Area', 'Prediction_Area'] + ['Probability_Action', 'Prediction_Action']

        return {'predictions': [{'fields': fields, 'values': values}]}

    return score

### AI function local test

Let's test defined AI function in notebook before we deploy it in the IBM Cloud. We start with preparing sample payload for `score` function.

In [15]:
sample_payload = {
    "input_data": [
        {
            "fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
            "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]
        }       
    ]
}

Let's test AI function locally before we deploy it to the IBM Cloud.

In [16]:
score = score_generator()
scores_ai = score(sample_payload)
print(scores_ai)

{'predictions': [{'fields': ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status', 'Car_Owner', 'Customer_Service', 'Satisfaction', 'Probability_Area', 'Prediction_Area', 'Probability_Action', 'Prediction_Action'], 'values': [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1, [0.4096774193548387, 0.29354838709677417, 0.04516129032258064, 0.06774193548387097, 0.06774193548387097, 0.041935483870967745, 0.054838709677419356, 0.01935483870967742], 'Service: Knowledge', [1.0, 0.0, 0.0, 0.0, 0.0], 'NA']]}]}


<a id="upload"></a>
## 3. Store the AI function in the repository

In this section we store AI function to watsonx.ai Runtime repository.

Required prerequisite to store AI function.

In [17]:
function_meta_props = {
            client.repository.FunctionMetaNames.NAME: 'CARS4U - Business area and Action Prediction - AI Function',
            client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
}

In [18]:
function_details = client.repository.store_function(score_generator, function_meta_props)

In [19]:
published_function_id = client.repository.get_function_id(function_details)

print("Function Id: " + str(published_function_id))

Function Id: 31255f31-50b9-4db3-b785-efd83889d858


In [20]:
client.repository.get_function_details(published_function_id)

{'entity': {'software_spec': {'id': '336b29df-e0e1-5e7d-b6a5-f6ab722625b2',
   'name': 'runtime-23.1-py3.10'},
  'type': 'python'},
 'metadata': {'created_at': '2024-03-06T12:21:46.888Z',
  'id': '31255f31-50b9-4db3-b785-efd83889d858',
  'modified_at': '2024-03-06T12:21:48.838Z',
  'name': 'CARS4U - Business area and Action Prediction - AI Function',
  'owner': 'IBMid-55000091VC',
  'space_id': '93ee84d1-b7dd-42b4-b2ca-121bc0c86315'},
 'system': {'warnings': []}}

<a id="deploy"></a>
## 4. Deploy AI function

In this section we deploy AI function in the IBM Cloud and test created deployment using sample payload.

In [ ]:
client.deployments.list()

In [21]:
deployment_details = client.deployments.create(
    published_function_id, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "CARS4U - Business area and Action Prediction - AI Function Deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)



#######################################################################################

Synchronous deployment creation for uid: '31255f31-50b9-4db3-b785-efd83889d858' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0107fd04-19a1-4892-b063-7aec404a1507'
------------------------------------------------------------------------------------------------




### AI function online deployment test

<a id="score"></a>
Now we are ready to test created deployment.

In [22]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 0]

In [23]:
import json

payload_scoring = {"input_data": [{"fields": fields,"values": [values]}]}
scoring_response = client.deployments.score(deployment_details['metadata']['id'], payload_scoring)

print(json.dumps(scoring_response, indent=3))

{
   "predictions": [
      {
         "fields": [
            "ID",
            "Gender",
            "Status",
            "Children",
            "Age",
            "Customer_Status",
            "Car_Owner",
            "Customer_Service",
            "Satisfaction",
            "Probability_Area",
            "Prediction_Area",
            "Probability_Action",
            "Prediction_Action"
         ],
         "values": [
            [
               3785,
               "Male",
               "S",
               1,
               17,
               "Inactive",
               "Yes",
               "The car should have been brought to us instead of us trying to find it in the lot.",
               0,
               [
                  0.4096774193548387,
                  0.29354838709677417,
                  0.04516129032258064,
                  0.06774193548387097,
                  0.06774193548387097,
                  0.041935483870967745,
                  0.054838709677

<a id="cleanup"></a>
## 5. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as watsonx.ai Runtime for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Amadeusz Masny**, Python Software Developer at watsonx.ai

**Mateusz Szewczyk**, Software Engineer at watsonx.ai

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.